## Day 9: Smoke Basin
[link](https://adventofcode.com/2021/day/9)

In [ ]:
#!value --name sampleRaw
2199943210
3987894921
9856789892
8767896789
9899965678

In [ ]:
#!value --name inputRaw --from-file ./data

In [ ]:

#!share sampleRaw --from value
#!share inputRaw --from value

In [ ]:
#load "../common.fsx"
#load "../formatting.fsx"

Formatter.Register<int[,]>((fun matrix -> formatTable (string) matrix), "text/html")
Formatter.Register<(int option)[,]>((fun matrix -> formatTable (function | None -> "." | Some value -> string value) matrix), "text/html")

let parse input = 
    read2d input
    |> Array2D.map (string >> int)
let sampleHeightMap = parse sampleRaw |> displayPipe
let actualHeightMap = parse inputRaw

2,1,9,9,9,4,3,2,1,0
3,9,8,7,8,9,4,9,2,1
9,8,5,6,7,8,9,8,9,2
8,7,6,7,8,9,6,7,8,9
9,8,9,9,9,6,5,6,7,8


In [ ]:
let lowPoints (heightMap : int[,]) = 
    Array2D.mapi (fun i j value ->
                            let adj = 
                                [|
                                    // use slicing opertator in order not to check boundaries.
                                    // each line returns at most one adjacent element
                                    heightMap.[i-1..i-1, j]
                                    heightMap.[i+1..i+1, j]
                                    heightMap.[i, j-1..j-1]
                                    heightMap.[i, j+1..j+1]
                                |] |> Array.collect id
                            
                            if (value < Array.min adj)
                            then Some value
                            else None
                        ) heightMap

lowPoints sampleHeightMap |> display
let riskLevel lowPointsMap = 
    lowPointsMap
    |> Array2D.toArray 
    |> Array.choose id
    |> Array.map ((+) 1)
    |> Array.sum
lowPoints sampleHeightMap |> riskLevel |> display
lowPoints actualHeightMap |> riskLevel |> display

.,1,.,.,.,.,.,.,.,0
.,.,.,.,.,.,.,.,.,.
.,.,5,.,.,.,.,.,.,.
.,.,.,.,.,.,.,.,.,.
.,.,.,.,.,.,5,.,.,.


15

417

In [ ]:
let (|Empty|NotEmpty|) set =
    if (Set.isEmpty set) then Empty
    else
        let head = Seq.head set
        NotEmpty (head, (Set.remove head set))
let basins (heightMap : int[,]) = 
    let allIndices = 
        heightMap |> Array2D.mapi (fun i j value -> match value with | 9 -> None | _ -> Some (i,j))
        |> Seq.cast<(int*int) option>
        |> Seq.choose id
        |> Set.ofSeq
        
    let rec findBasin i j indicesLeft basinAcc =
        if (not <| Set.contains (i,j) indicesLeft) then indicesLeft, basinAcc
        else 
            let newIndices = indicesLeft |> Set.remove (i,j)
            (newIndices, (i,j)::basinAcc)
            ||> findBasin (i-1) j
            ||> findBasin i (j-1)
            ||> findBasin (i+1) j
            ||> findBasin i (j+1)

    let rec find (indicesLeft:Set<int*int>) (basinsAcc : (int*int) list list) =
        if (Set.isEmpty indicesLeft) then basinsAcc
        else
            let (i,j) = Seq.head indicesLeft
            let (newIndicesLeft, basin) = findBasin i j indicesLeft []
            basin :: (find newIndicesLeft [])
     
    find allIndices [] |> List.filter (not << List.isEmpty)

In [ ]:
let private showBasins (basins: (int*int) list list) (heightMap: int[,]) = 
    let basinsMap = Array2D.zeroCreate (Array2D.length1 heightMap) (Array2D.length2 heightMap)
    basins
    |> Seq.iteri (fun num basin -> basin |> Seq.iter (fun (i,j) -> basinsMap.[i,j] <- Some num))
    basinsMap |> display |> ignore

showBasins (basins sampleHeightMap) sampleHeightMap

0,0,.,.,.,1,1,1,1,1
0,.,2,2,2,.,1,.,1,1
.,2,2,2,2,2,.,3,.,1
2,2,2,2,2,.,3,3,3,.
.,2,.,.,.,3,3,3,3,3


In [ ]:
let private multiplyTop3 basins = 
    basins
    |> Seq.sortByDescending List.length
    |> Seq.take 3
    |> Seq.map List.length
    |> Seq.reduce (*)

multiplyTop3 (basins sampleHeightMap) |> display
multiplyTop3 (basins actualHeightMap) |> display

1134

1148965